In [8]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth
from sklearn.cluster import SpectralClustering, AffinityPropagation, MiniBatchKMeans
from sklearn import metrics
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
%matplotlib inline

In [40]:
import os
curr_directory = os.getcwd()
datafile = curr_directory + "/all_recipe_data.json"

In [69]:
import json
raw_recipe_data = pd.read_json(datafile)

In [52]:
raw_recipe_data.isnull().values.any()

False

In [70]:
raw_recipe_data.Instructions.replace('', np.nan, inplace=True)
raw_recipe_data.head()

,Author,Instructions
0,[witchywoman],"Mix grapefruit, orange, apples, pineapple, mar..."
1,[witchywoman],Fill a margarita glass with crushed ice. Pour ...
2,[witchywoman],Preheat an oven to 350 degrees F (175 degrees ...
3,[witchywoman],
4,[witchywoman],"Mix the teriyaki sauce, garlic, ginger, and re..."


In [76]:
raw_recipe_data.isnull().sum()

Author          0
Instructions    0
dtype: int64

In [ ]:
# import modules & set up logging
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [105]:
sentences = []

for text in raw_recipe_data.Instructions:
    sents = text.lower().split('.')
    sentences.append(sents)

In [114]:
np.random.shuffle(sentences)
training, test = sentences[:81], sentences[81:]


In [115]:
len(test)

26